In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from numba import njit
from tqdm.notebook import tqdm

# Fast Deterministic CUR Matrix Decomposition with Accuracy Assurance

In [24]:
# import numpy as np
# from copy import deepcopy
# from numba import njit
# from matplotlib import pyplot as plt
# from tqdm.notebook import tqdm
# # Your solution is here
# eps=1e-3
# N = 12
# xi = np.array([i/(N-1) for i in range(N)])
# xj = np.array([i/(N-1) for i in range(N)])
# xx, yy = np.meshgrid(xi, xj)
# A = np.exp(-np.sqrt(xx**2 + yy**2))
# C, U, R = CUR(X=A, lmb=11.15, eps=eps)
# A_CUR = C@np.linalg.pinv(U)@R
# # A_CUR, C, U, R = CUR_naive(A,1)
# print(np.linalg.norm(A-A_CUR))
# print(C.shape)
# print(U.shape)
# print(R.shape)
# print(A_CUR.shape)

In [2]:
###Compute W according to Algorithm 1
#Auxuluary function for computing optimality score
def compute_XW(X, W, i):
    n, p = X.shape
    XW = np.zeros((n, p))
    for j in range(p):
        if i != j:
            XW += np.outer(X[:,[j]], W[[j], :])
    return XW

#COORDINATE DESCENT for CUR deterministic problem
def coordinate_descent(X, A, W, lmb, eps):
    """
    X - array of the matrix to decompose;
    A - set of column indices to iterate
    W - matrix of parameters
    lmb - regularization coefficient of the optimization problem
    eps - convergence tolerance 
    """
    W = W.copy()
    X = X.copy()
    n, p = X.shape
    W_new = W.copy()
    if len(A) == 0:
        print('Empty set of indices')
        return W
    
    while True:
        for i in A:
            XW = compute_XW(X=X, W=W, i=i)
            z_i = X[:,[i]].T @ (X - XW) #eq(4)
            assert z_i.shape == (1, p), "dimensionality error"
            norm_z = np.linalg.norm(z_i)
            
            step = (1 - lmb/norm_z)
            step = step if step > 0.0 else 0.0 #eq(3)

            W_new[i,:] = step * z_i #eq(2)
            
        if np.linalg.norm(W_new - W, ord='fro') < eps:
            W = W_new
            break
        else:
            W = W_new
    return W

#return CUR matrices computed by algorithm 1
def CUR(X, lmb, eps):
    n, p = X.shape
    #col search
    W = np.zeros((p, p))
    A = np.arange(p)
    W_col = coordinate_descent(X=X, A=A, W=W, lmb=lmb, eps=eps)
    
    #row search
    W = np.zeros((n, n))
    A = np.arange(n)
    X = X.T.copy()
    W_row = coordinate_descent(X=X, A=A, W=W, lmb=lmb, eps=eps)

    #extract indices
    X = X.T.copy()
    Icol = np.argwhere(np.any(W_col, axis=1)).ravel()
    Irow = np.argwhere(np.any(W_row, axis=1)).ravel()
    
    #extract CUR
    C = X[:, Icol]
    R = X[Irow, :]
    II, JJ = np.meshgrid(Irow, Icol)
    U = X[II, JJ].T

    return C, U, R


In [3]:
#RBF matrix for R+
N = 100
xi = np.array([i/(N-1) for i in range(N)])
xj = np.array([i/(N-1) for i in range(N)])
xx, yy = np.meshgrid(xi, xj)
A_rbf = np.exp(-np.sqrt(xx**2 + yy**2))

In [13]:
#Parameters
eps = 1e-3
lmb_max = 7 #Can be tuned with condition: smallest lmb when all W are 0
Q = 10

np.random.seed(10)
n, p = (6, 4)
X = A_rbf#np.random.randn(n,p)
X = X / np.linalg.norm(X, axis=0)

C, U, R = CUR(X=X, lmb=lmb_max, eps=eps)
print(C.shape, U.shape, R.shape, )
print(C[0], U[0], R[0])
print()

print(X)
print()

X_ = C@np.linalg.pinv(U)@R
print(X_)
print()

print(np.linalg.norm(X - X_, ord='fro'))

(100, 96) (90, 96) (90, 100)
[0.15184663 0.14567241 0.14465343 0.14368585 0.14276461 0.14188548
 0.14104486 0.14023966 0.13946719 0.13872509 0.13801126 0.13732383
 0.13666112 0.13602163 0.13540399 0.13480694 0.13422935 0.13367019
 0.13312848 0.13260336 0.132094   0.13159964 0.13111958 0.13065318
 0.13019981 0.12975891 0.12932995 0.12891243 0.12850587 0.12810984
 0.12772391 0.1273477  0.12698084 0.12662297 0.12627376 0.12593291
 0.1256001  0.12527507 0.12495753 0.12464723 0.12434394 0.12404741
 0.12375742 0.12347377 0.12319625 0.12292467 0.12265885 0.1223986
 0.12214375 0.12189415 0.12164964 0.12141007 0.1211753  0.12094518
 0.12071959 0.12049839 0.12028147 0.12006871 0.11985999 0.1196552
 0.11945424 0.11925701 0.11906341 0.11887333 0.1186867  0.11850342
 0.11832341 0.11814659 0.11797287 0.11780217 0.11763443 0.11746957
 0.11730752 0.11714822 0.11699159 0.11683757 0.11668611 0.11653714
 0.11639061 0.11624645 0.11610462 0.11596505 0.11582771 0.11569254
 0.11555948 0.11542851 0.11529956 0

In [14]:
print(C, )
print()
print(U, )
print()
print(R)

[[0.15184663 0.14567241 0.14465343 ... 0.11445015 0.11433593 0.11422338]
 [0.15032055 0.14552679 0.14453255 ... 0.11444419 0.11433004 0.11421756]
 [0.1488098  0.14510677 0.14417998 ... 0.11442631 0.11431236 0.11420008]
 ...
 [0.05700124 0.05744148 0.05758581 ... 0.07627107 0.07641931 0.07656611]
 [0.05642837 0.05686493 0.05700815 ... 0.07572686 0.07587682 0.07602535]
 [0.05586125 0.05629416 0.05643626 ... 0.0751838  0.07533544 0.07548564]]

[[0.15184663 0.14567241 0.14465343 ... 0.11445015 0.11433593 0.11422338]
 [0.15032055 0.14552679 0.14453255 ... 0.11444419 0.11433004 0.11421756]
 [0.1488098  0.14510677 0.14417998 ... 0.11442631 0.11431236 0.11420008]
 ...
 [0.06242602 0.06289979 0.06305417 ... 0.08120797 0.08133835 0.08146725]
 [0.06179862 0.06226864 0.06242191 ... 0.08065698 0.08078956 0.08092066]
 [0.06117754 0.0616438  0.06179596 ... 0.0801064  0.08024111 0.08037436]]

[[0.15184663 0.15041912 0.1491094  ... 0.11445015 0.11433593 0.11422338]
 [0.15032055 0.14979109 0.14875427 ..

In [15]:
#Compute W according to Algorithm 2
def FastCUR(X, 
            lmb_max=10, gamma=4, Q=10, eps=1e-3, 
            fixed_lmb=False, params=True):
    
    X = X.copy()
    n, p = X.shape
    if fixed_lmb == True:
        Q_lmb = [lmb_max] * Q
    else:
        Q_lmb = np.array([10**(-gamma * q / (Q - 1)) \
                          for q in range(Q)]) * lmb_max
    
    #line 1-3
    A = np.arange(p)
    G = (X.T)@X
    G_norms = np.array([np.linalg.norm(G[i,:]) for i in range(p)])
    W = np.zeros((p, p))
    W_tilde = np.zeros((p, p))
    # W_new is used for inner cycle termination
    W_new = np.ones((p, p)) * np.inf
    
    K_tilde = G_norms.copy()
    K_u = K_tilde.copy()
    K_l = np.zeros(p)
    
    delta = 0.0 #zero, since W = W_tilde = 0
    ##flag = 0
    
    M_global = []
    for it, lmb in enumerate(Q_lmb):
        print('iter:', it)
        M = []
        dW = W - W_tilde
        dW_norms = np.array([np.linalg.norm(dW[i,:]) for i in range(p)])
        dW_fro = np.linalg.norm(dW, ord='fro')

        for i in range(p):
            if it == 0:
                K_l = K_tilde
            else:
                K_l[i] = K_u[i] - 2 * (dW_norms[i] + delta * G_norms[i]) #eq(10)
            
            if K_l[i] > lmb:
                M.append(i)
        print(np.linalg.norm(W, ord='fro'))
        W = coordinate_descent(X=X, A=M, W=W, lmb=lmb, eps=eps)
        print(np.linalg.norm(W, ord='fro'))
        while True:
            if np.linalg.norm(W_new - W, ord='fro') < eps:
                flag = 1
                break
            else:
                W_new = W

            W_tilde = W.copy() #line 15
            #cycle 16-17 - update K_tilde
            for i in A:
                z_i = X[:,[i]].T @ (X - compute_XW(X, W_tilde, i))
                K_tilde[i] = np.linalg.norm(z_i)
                
            for i in A:
                dW_i = dW[i,:] #W[i,:] - W_tilde[i,:]
                K_u[i] = K_tilde[i] + np.linalg.norm(dW_i) + delta * G_norms[i]
                
                if K_u[i] <= lmb:
                    W_prime = 0.0
                else:
                    z_i = X[:,[i]].T @ (X - compute_XW(X, W, i))
                    norm_z = np.linalg.norm(z_i)
                    step = (1 - lmb/norm_z) 
                    step = step if step > 0.0 else 0.0 #eq(3)
                    W_prime = step * z_i #eq2

                delta = np.sqrt(np.abs(dW_fro**2 - \
                                np.linalg.norm(dW_i)**2 + \
                                np.linalg.norm(W_prime - W_tilde[i,:])**2))
                
                W[i,:] = W_prime
        
        print(np.linalg.norm(W, ord='fro'))
        print('Number of nonzero rows:', np.sum(np.all(W, axis=1)))
        M_global.append(M)
        #Global stop the algorithm
        if np.all(W)==True:
            print('Converges in {0} iteration'.format(it+1))
            break
    if params:
        return W, M_global, Q_lmb[:it+1]
    else:
        return W


In [52]:
#Parameters
eps = 1e-3
lmb_max = 1.25 #18 #Can be tuned with condition: smallest lmb when all W are 0
gamma = 0.05
Q = 20

np.random.seed(10)
n, p = (220, 100)
X = np.random.randn(n,p) #np.ones((n,p)) #A_rbf # #A_rbf #
X = X / np.linalg.norm(X, axis=0)
n, p = X.shape
#X = np.ones()
#X[:,1] = np.random.randn(n)


print(X)
print()

#TEST the old alg
W = np.zeros((p, p))
A = np.arange(p)
%time W_old = coordinate_descent(X, A, W, lmb=lmb_max, eps=eps)
print(W_old)
print()
print('Number of nonzero rows:', np.sum(np.all(W_old, axis=1)))
print()

#TEST the new alg
%time W_fast, M_, Q_ = FastCUR(X, lmb_max=lmb_max, gamma=gamma, Q=Q, eps=eps, fixed_lmb=False)
print(W_fast)
print()


print()
print('Are they the same?:', np.all(np.isclose(W_old, W_fast)))

[[ 0.09133403  0.04618891 -0.10227573 ...  0.00609735  0.05450841
  -0.13562388]
 [ 0.0080577  -0.12317341 -0.06107881 ... -0.06957869  0.07692939
  -0.03968955]
 [ 0.00913195  0.07766679 -0.0678189  ... -0.00987312  0.01080311
   0.05755999]
 ...
 [-0.0156005   0.00361708 -0.07517762 ...  0.10692958 -0.03868686
   0.03093576]
 [ 0.1200547  -0.02142421  0.08926696 ...  0.05631732  0.07587396
   0.08062387]
 [ 0.06008339 -0.11729766  0.07437738 ...  0.08513894  0.02745995
   0.02189656]]

CPU times: user 6.02 s, sys: 144 ms, total: 6.16 s
Wall time: 1.57 s
[[ 0.  0.  0. ...  0. -0. -0.]
 [ 0.  0.  0. ... -0. -0.  0.]
 [ 0.  0.  0. ... -0.  0.  0.]
 ...
 [ 0. -0. -0. ...  0. -0.  0.]
 [-0. -0.  0. ... -0.  0.  0.]
 [-0.  0.  0. ...  0.  0.  0.]]

Number of nonzero rows: 2

iter: 0
0.0
0.025911226451066537
0.02592357659878312
Number of nonzero rows: 2
iter: 1
0.02592357659878312
0.03531349359897828
0.03532736839785541
Number of nonzero rows: 4
iter: 2
0.03532736839785541
0.046784331588821

In [54]:
M_

[[11, 41, 64],
 [11, 41, 64, 91],
 [11, 13, 41, 64, 82, 91],
 [10, 11, 13, 41, 45, 64, 81, 82, 91],
 [10, 11, 13, 16, 37, 41, 45, 62, 64, 73, 76, 81, 82, 91, 94],
 [8,
  10,
  11,
  13,
  16,
  17,
  25,
  29,
  37,
  41,
  45,
  48,
  49,
  51,
  58,
  60,
  62,
  64,
  68,
  73,
  74,
  76,
  81,
  82,
  88,
  89,
  91,
  94],
 [3,
  8,
  10,
  11,
  13,
  16,
  17,
  24,
  25,
  29,
  30,
  32,
  37,
  41,
  45,
  48,
  49,
  51,
  58,
  60,
  62,
  64,
  65,
  68,
  70,
  73,
  74,
  76,
  81,
  82,
  88,
  89,
  91,
  94,
  98],
 [3,
  8,
  10,
  11,
  13,
  16,
  17,
  24,
  25,
  29,
  30,
  32,
  36,
  37,
  41,
  45,
  48,
  49,
  51,
  58,
  60,
  62,
  64,
  65,
  68,
  70,
  73,
  74,
  76,
  81,
  82,
  88,
  89,
  91,
  94,
  96,
  98],
 [3,
  8,
  10,
  11,
  13,
  16,
  17,
  24,
  25,
  26,
  29,
  30,
  32,
  34,
  36,
  37,
  38,
  41,
  45,
  48,
  49,
  51,
  52,
  55,
  58,
  60,
  61,
  62,
  64,
  65,
  68,
  70,
  71,
  73,
  74,
  76,
  81,
  82,
  83,
  88,
 

In [53]:
Q_

array([1.25      , 1.24244861, 1.23494284, 1.22748241, 1.22006705,
       1.21269648, 1.20537045, 1.19808867, 1.19085088, 1.18365681,
       1.17650621, 1.1693988 , 1.16233433, 1.15531254, 1.14833316,
       1.14139595, 1.13450065, 1.12764701, 1.12083476, 1.11406367])

In [ ]:
from cur import cur_decomposition
C, U, R = cur_decomposition(X, 3, )

In [ ]:
#C @ np.linalg.pinv(U) @ R == X

array([[-2.29931842, -2.29931842, -0.72913767],
       [ 0.27332231,  0.27332231, -0.80748006],
       [-0.41226214, -0.41226214, -1.33354773],
       [-0.97011725, -0.97011725, -1.96449257],
       [-0.59226263, -0.59226263, -0.04675949]])

In [ ]:
mycur(X, 1)

NameError: ignored

In [140]:
# https://www.programmersought.com/article/69322038039/

def CUR(A, n): 
    A_sq = A ** 2
    sum_A_sq = np.sum(A_sq)
    sum_A_sq_0 = np.sum(A_sq, axis=0)
    sum_A_sq_1 = np.sum(A_sq, axis=1)
    
    P_x_c = sum_A_sq_0 / sum_A_sq
    P_x_r = sum_A_sq_1 / sum_A_sq
    
    r, c = A.shape
    
    c_index = [np.random.choice(np.arange(0, c), p=P_x_c) for i in range(n)]
    r_index = [np.random.choice(np.arange(0, r), p=P_x_r) for i in range(n)]
#     print(c_index, r_index)
    C = A[:, c_index]
    R = A[r_index, :]
    W = C[r_index]
#     print(C, R, W)
    
    
    def SVD(A, n):
        M = np.dot(A, A.T)
        eigval, eigvec = np.linalg.eig(M)
        indexes = np.argsort(-eigval)[:n]
        U = eigvec[:, indexes]
        sigma_sq = eigval[indexes]
        M = np.dot(A.T, A)
        eigval, eigvec = np.linalg.eig(M)
        indexes = np.argsort(-eigval)[:n]
        V = eigvec[:, indexes]
        sigma = sigma_sq # not diag and not sqrt
        return U, sigma, V
    
    X, sigma, Y = SVD(W, n)
    for i in range(len(sigma)):
        if sigma[i] == 0:
            continue
        else:
            sigma[i] = 1 / sigma[i]
    sigma = np.diag(sigma)
    U = np.dot(np.dot(Y, sigma), X.T)
    return C, U, R

In [142]:
CUR(X, 3)

(array([[0.63473642, 0.56227874, 0.71653131],
        [0.63466519, 0.56222892, 0.71642168],
        [0.63445167, 0.56207955, 0.7160932 ],
        [0.6340963 , 0.56183084, 0.71554706],
        [0.63359986, 0.56148315, 0.71478525],
        [0.6329634 , 0.56103698, 0.7138105 ],
        [0.63218825, 0.56049297, 0.71262626],
        [0.63127606, 0.5598519 , 0.71123666],
        [0.63022869, 0.55911469, 0.70964643],
        [0.62904831, 0.55828236, 0.70786088],
        [0.62773727, 0.55735608, 0.7058858 ],
        [0.62629819, 0.55633713, 0.70372742],
        [0.62473386, 0.5552269 , 0.70139233],
        [0.62304728, 0.55402689, 0.69888742],
        [0.6212416 , 0.55273869, 0.6962198 ],
        [0.61932012, 0.55136399, 0.69339677],
        [0.61728628, 0.54990458, 0.69042572],
        [0.61514361, 0.5483623 , 0.68731412],
        [0.61289574, 0.54673907, 0.68406942],
        [0.61054639, 0.5450369 , 0.68069904],
        [0.60809932, 0.54325783, 0.67721034],
        [0.60555832, 0.54140395, 0